<a href="https://colab.research.google.com/github/rswijesena/AI/blob/main/OpenAI_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install panda
!pip install pip install langchain


In [2]:
import openai

In [3]:
openAIKey=""

In [ ]:
openai.api_key = openAIKey
all_models = openai.models.list()
list(all_models)


In [ ]:
import pandas as pd
pd.DataFrame(list(all_models))

In [6]:
from openai import OpenAI
client = OpenAI(api_key=openAIKey)

In [ ]:
from openai import OpenAI

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": "company assistant"
    },
    {
      "role": "user",
      "content": "who won the first cricket world cup?"
    }
  ],
  temperature=1,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
response.choices[0].message.content

In [8]:
import langchain

In [9]:
developer_description = "Roshan Wijsena is a integration developer at Boomi with 10 years of industry experince, he studied at UCD and he is in gravonder club, his grade is grade A"



In [10]:
from openai import OpenAI

prompt = f'''
Please extract the following information from the given text and return it as a JSON object:

name
collage
grades
club

This is the body of text to extract the information from:
{developer_description}
'''

messages = [
    {
        "role": "user",
        "content": prompt
    }
]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages
)

print(response.choices[0].message.content)

{
  "name": "Roshan Wijsena",
  "college": "UCD",
  "grades": "grade A",
  "club": "gravonder club"
}


In [13]:
## Calling function

from openai import OpenAI
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
print(list(run_conversation()))

[('id', 'chatcmpl-9OEXnMIjXIHRCykmjZKCbwLKh9UV9'), ('choices', [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current weather in San Francisco is 72°F, in Tokyo it is 10°C, and in Paris it is 22°C.', role='assistant', function_call=None, tool_calls=None))]), ('created', 1715562807), ('model', 'gpt-3.5-turbo-0125'), ('object', 'chat.completion'), ('system_fingerprint', None), ('usage', CompletionUsage(completion_tokens=28, prompt_tokens=147, total_tokens=175))]


Advance Function calling

In [17]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {
            "role": "user",
            "content": "When's the next flight from delhi to mumbai?"
        }
    ],
)
response.choices[0].message.content

'There are multiple flights from Delhi to Mumbai throughout the day. It is recommended to check with airlines or travel websites for the most up-to-date and accurate information on flight schedules.'

In [ ]:
function_description = [
    {
        "name": "get_next_flight",
        "description": "Get the next flight from delhi to mumbai",
        "parameters": {
            "type": "object",
            "properties": {
                "from": {"type": "string"},
                "to": {"type": "string"},
                "date": {"type": "string"},
            },
            "required": ["from", "to", "date"],
        },
    }
]